# Install detectron2

In [5]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

/bin/bash: python: 명령어를 찾을 수 없음
/bin/bash: python: 명령어를 찾을 수 없음


In [1]:
import torch, detectron2
import os
import sys
ROOT_DIR = os.path.abspath("./")
# Import Mask RCNNon of the library
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


/home/agtech-research/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
import xml.etree.ElementTree as ET
import shutil
from detectron2.structures import BoxMode

def get_flower_dicts(img_dir, mode):
    dataset_dicts = []
    fname=[]
    fn = []
    i_dir = "/home/agtech-research/바탕화면/strawberry/help/dataset/02_output/"
    if mode == 'train':
        fname.append(os.path.join(i_dir, "KETI_Strawberry_2.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_3.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_4.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_5.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_6.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_9.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_10.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_11.xml"))
        fname.append(os.path.join(i_dir, "KETI_Strawberry_12.xml"))
        fn = [2,3,4,5,6,9,10,11,12]

    elif mode == 'val':
        fname.append(os.path.join(i_dir, "KETI_Strawberry_1.xml"))
        fn=[1]

    else :
        fname.append(os.path.join(i_dir, "KETI_Strawberry_7.xml"))
        fn=[7]
        
    for i in range(len(fname)):
        anno_doc = ET.parse(fname[i])
        annoD_root = anno_doc.getroot()
        for items in annoD_root.iter("image"):
            filename = items.attrib["name"]
            boxlist = items.findall("box")
            
            
            record={}
            record["file_name"] = img_dir+str(fn[i])+"/"+filename
            #print(record["file_name"])
            record["image_id"] = items.attrib["id"]
            record["height"] = int(items.attrib["height"])
            record["width"] = int(items.attrib["width"])

            #print(boxlist)

            objs = []
            for bidx in range(len(boxlist)): 
                #xml에서 박스 찾기 및 저장된 속성값 가져오기
                #occ = boxlist[bidx].attrib["occluded"] #사용은 안함
                xmax= boxlist[bidx].attrib["xbr"]
                xmin= boxlist[bidx].attrib["xtl"]
                ymax= boxlist[bidx].attrib["ybr"]
                ymin= boxlist[bidx].attrib["ytl"]    
                #z= boxlist[bidx].attrib["z_order"] #사용은 안함 

                px = [float(xmax), float(xmin)]
                py = [float(ymax), float(ymin)]   

                obj = {
                        "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                        "bbox_mode": BoxMode.XYXY_ABS,
                        #"segmentation": [],
                        "category_id": 0,
                }
                objs.append(obj)

            record["annotations"] = objs
            dataset_dicts.append(record)
    return dataset_dicts

DatasetCatalog.clear()
for d in ["train", "val","test"]:
    DatasetCatalog.register("flower_" + d, lambda d=d: get_flower_dicts("/home/agtech-research/바탕화면/strawberry/help/dataset/01_rawData/KETI_Strawberry_",d))
    MetadataCatalog.get("flower_" + d).set(thing_classes=["flower"])

flower_metadata = MetadataCatalog.get("flower_train")

In [5]:
"""dataset_dicts = get_flower_dicts("./KETI_Strawberry_1/",'1') 
#print(dataset_dicts)
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    #print(d["file_name"])
    v = Visualizer(img[:, :, ::-1], metadata=flower_metadata, scale=0.5)
    #print(" ",d)
    out = v.draw_dataset_dict(d)
    cv2.imshow('',out.get_image()[:, :, ::-1])"""


'dataset_dicts = get_flower_dicts("./KETI_Strawberry_1/",\'1\') \n#print(dataset_dicts)\nfor d in random.sample(dataset_dicts, 1):\n    img = cv2.imread(d["file_name"])\n    #print(d["file_name"])\n    v = Visualizer(img[:, :, ::-1], metadata=flower_metadata, scale=0.5)\n    #print(" ",d)\n    out = v.draw_dataset_dict(d)\n    cv2.imshow(\'\',out.get_image()[:, :, ::-1])'

In [4]:
from detectron2.utils.logger import setup_logger
setup_logger()

import os
from detectron2.engine import DefaultTrainer
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper, build_detection_train_loader, build_detection_test_loader   # the default mapper
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
from detectron2.engine.hooks import HookBase
from detectron2.utils.logger import log_every_n_seconds
import detectron2.utils.comm as comm
import time
import datetime
import logging

class LossEvalHook(HookBase):
  def __init__(self, eval_period, model, data_loader):
    self._model = model
    self._period = eval_period
    self._data_loader = data_loader

  def _do_loss_eval(self):
    # copying inference_on_dataset from evaluator.py
    total = len(self._data_loader)
    num_warmup = min(5, total -1)

    start_time = time.perf_counter()
    total_compute_time = 0
    losses =[]

    for idx, inputs in enumerate(self._data_loader):
      if idx == num_warmup:
        start_time = time.perf_counter()
        total_compute_time = 0
      start_compute_time = time.perf_counter()
      if torch.cuda.is_available():
        torch.cuda.synchronize()
      total_compute_time += time.perf_counter()- start_compute_time
      iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
      seconds_per_image = total_compute_time / iters_after_start
      if idx >= num_warmup * 2 or seconds_per_image > 5:
        total_seconds_per_image = (time.perf_counter() - start_time) /iters_after_start
        eta = datetime.timedelta(seconds=int(total_seconds_per_image * (total - idx - 1)))
        log_every_n_seconds(
            logging.INFO,
            "Loss on Validation done {}/{}. {:.4f} s / img. ETA={}".format(
                idx + 1, total, seconds_per_image, str(eta)
            ),
            n=5,
        )
        loss_batch = self._get_loss(inputs)
        losses.append(loss_batch)
      mean_loss = np.mean(losses)
      self.trainer.storage.put_scalar('validation_loss',mean_loss)
      comm.synchronize()

      return losses

    def _get_loss(self, data):
      # How loss is calculated on trainloop 
      metrics_dict = self._model(data)
      metrics_dict = {
          k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
          for k, v in metrics_dict.items()
      }
      total_losses_reduced = sum(loss for loss in metrics_dict.values())
      return total_losses_reduced

    def after_step(self):
      next_iter = self.trainer.iter +1
      is_final = next_iter == self.trainer.max_iter
      if is_final or (self._period > 0 and next_iter % self._period ==0):
        self._do_loss_eval()

      self.trainer.storage.put_scalars(timetest = 12)



# use this dataloader instead of the default

class CustomTrainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains a number pre-defined logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can use the cleaner
    "SimpleTrainer", or write your own training loop.
    """
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
      if output_folder is None :
        output_folder = "/home/agtech-research/바탕화면/strawberry/help/detectron2/output/model_outputs_"+str(cfg.SOLVER.BASE_LR)+"_"+str(cfg.SOLVER.MAX_ITER)
      return COCOEvaluator(dataset_name, cfg, True, output_folder)

    def build_hooks(self):
      hooks = super().build_hooks()
      hooks.insert(-1, LossEvalHook(
          cfg.TEST.EVAL_PERIOD,
          self.model,
          build_detection_test_loader(
              self.cfg,
              self.cfg.DATASETS.TEST[0],
              mapper = DatasetMapper(self.cfg,True, augmentations=[ T.Resize((640, 640)) ])
          )
      ))
      return hooks

    @classmethod
    def build_train_loader(cls, cfg):
        #dataloader = build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[ T.Resize((640, 640)) ]))
        print("커스텀 사용됨!")
        mapper=DatasetMapper(cfg, is_train=True, augmentations=[ T.Resize((640, 640)) ])
        return build_detection_train_loader(cfg, mapper= mapper)
        

In [5]:
root = "/home/agtech-research/바탕화면/strawberry/help/detectron2/configs/"
cfg = get_cfg()
cfg.merge_from_file(root+"COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("flower_train",)
cfg.DATASETS.TEST =  ("flower_val",)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x")   # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people,
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500
cfg.TEST.EVAL_PERIOD = 100 #몇 iter마다 validation 할 것인지 


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (flower).

cfg.OUTPUT_DIR="/home/agtech-research/바탕화면/strawberry/help/detectron2/output/model_outputs_"+str(cfg.SOLVER.BASE_LR)+"_"+str(cfg.SOLVER.MAX_ITER)+"_"+str(cfg.TEST.EVAL_PERIOD)



os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[08/01 14:58:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

In [7]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
#outputs = predictor(cfg)

[08/01 14:59:47 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [8]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_flower_dicts("image_dir",'val')
for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print(outputs)
    v = Visualizer(im[:, :, ::-1],
                   metadata=flower_metadata, 
                   scale=0.5, 
                    # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    #print(outputs["instances"])
    #out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    #cv2.imshow(" ",out.get_image()[:, :, ::-1])

[ WARN:0@189.948] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('image_dir1/20191127_F_A-1_3.JPG'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [12]:
# inference time 측정
import time

dataset_dicts = get_flower_dicts("/home/agtech-research/바탕화면/strawberry/help/dataset/01_rawData/KETI_Strawberry_",'test')
flag = 0
for d in dataset_dicts :  
    if flag == 0 : 
        start = time.time()
        flag = 1 
    before_read = time.time() 
    im = cv2.imread(d["file_name"])
    after_read = time.time()
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    end = time.time()
    #print(outputs)
    print("b-end :",end - before_read) #평균 0.1
    
    
print("FPS: ", 193.0 / (end - start)) # 약 0.104
print((end - start)/60)
    

b-end : 0.09870100021362305
b-end : 0.0989692211151123
b-end : 0.10032463073730469
b-end : 0.09951257705688477
b-end : 0.09781813621520996
b-end : 0.09846115112304688
b-end : 0.0997769832611084
b-end : 0.09813666343688965
b-end : 0.09778261184692383
b-end : 0.09917163848876953
b-end : 0.09935665130615234
b-end : 0.09914112091064453
b-end : 0.09839987754821777
b-end : 0.09906840324401855
b-end : 0.09741425514221191
b-end : 0.10317707061767578
b-end : 0.09901189804077148
b-end : 0.09861564636230469
b-end : 0.09951186180114746
b-end : 0.09824156761169434
b-end : 0.09892392158508301
b-end : 0.09811830520629883
b-end : 0.09811568260192871
b-end : 0.09911394119262695
b-end : 0.09913992881774902
b-end : 0.0987997055053711
b-end : 0.09800887107849121
b-end : 0.09972739219665527
b-end : 0.0978245735168457
b-end : 0.09841752052307129
b-end : 0.10141873359680176
b-end : 0.09929060935974121
b-end : 0.10191798210144043
b-end : 0.10025572776794434
b-end : 0.09858393669128418
b-end : 0.09960556030273

In [9]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("flower_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "flower_test", mapper = DatasetMapper(cfg,True, augmentations=[ T.Resize((640, 640)) ]))
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[08/01 13:15:37 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[08/01 13:15:37 d2.evaluation.coco_evaluation]: Trying to convert 'flower_test' to COCO format ...
[08/01 13:15:37 d2.data.datasets.coco]: Converting annotations of dataset 'flower_test' to COCO format ...)
[08/01 13:15:37 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[08/01 13:15:37 d2.data.datasets.coco]: Conversion finished, #images: 95, #annotations: 196
[08/01 13:15:37 d2.data.datasets.coco]: Caching COCO format annotations at './output/flower_test_coco_format.json' ...
[08/01 13:15:37 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [Resize(shape=(640, 640))]
[08/01 13:15:37 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   flower   | 196          |
|            |              |
[08/01 13:15:37 d2.data.common]: Serializing 95 elements to byte tens